In [0]:
import numpy as np
import tensorflow as tf
import pickle

adc.json	mnist_train.csv  saved_model
mnist_test.csv	sample_data	 saved_model.tar.gz


In [0]:
# downloading and loading images
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

def gdrive():
    global drive
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)

In [0]:
downloaded = drive.CreateFile({'id': "1hydbOxr3GLmm5_zaGOfshvml5fYY_P8a"})
downloaded.GetContentFile("train.pkl")
print "train"
gdrive()
downloaded = drive.CreateFile({'id': "1tXK20fHWydieLNhDiQo39ae0lfTxF3pa"})
downloaded.GetContentFile("validation.pkl")
print "validation"
gdrive()
downloaded = drive.CreateFile({'id': "1Xp4xV6yAqYpTMfOQIQ4wkDk8sZT7fHeN"})
downloaded.GetContentFile("test.pkl")
print "test"
gdrive()
downloaded = drive.CreateFile({'id': "1QDQxjVPxcDw7mrzMxep2OYCFkh8ElbSA"})
downloaded.GetContentFile("new.pkl")
print "new"

1
validation
1.pkl  adc.json  sample_data  validation.pkl


In [0]:
train = True
test = True
batch_size = 36
height = 101
width = 101
image_channels = 3
num_classes= 2
learn_rate = 1e-3

image_batch = tf.placeholder(dtype = tf.float32,shape=[batch_size,height,width,image_channels])
masks = tf.placeholder(dtype=tf.float32 ,shape=[batch_size,height,width,num_classes])
is_training = tf.placeholder(dtype=tf.bool,shape=[])
learning_rate = tf.placeholder(dtype=tf.float32,shape=[])

#standardize the input
images = tf.map_fn(lambda frame: tf.image.per_image_standardization(frame), image_batch)
print images



Tensor("map/TensorArrayStack/TensorArrayGatherV3:0", shape=(36, 101, 101, 3), dtype=float32)


In [0]:
#7x7 convolution stride 2 to downsample by factor 2
conv1 = tf.layers.conv2d(images,
                         filters=64,
                         kernel_size = 7,
                         strides=(2,2),
                         padding='valid',
                         activation=None,
                         kernel_initializer=tf.contrib.layers.xavier_initializer())
# conv1= tf.layers.batch_normalization(conv1,training=True)
print conv1
conv1_bnr = tf.contrib.layers.batch_norm(conv1, activation_fn=tf.nn.relu,is_training = is_training)
print conv1_bnr

Tensor("conv2d/BiasAdd:0", shape=(36, 48, 48, 64), dtype=float32)
Tensor("BatchNorm/Relu:0", shape=(36, 48, 48, 64), dtype=float32)


In [0]:
conv2_blocks = 3
unactivated = conv1
activated = conv1_bnr
for i in range(conv2_blocks):
    with tf.variable_scope("block2_"+str(i)):
        conv_0 = tf.layers.conv2d(activated,
                                   filters=128/2,
                                   kernel_size = 1,
                                   strides=(1,1),
                                   padding='same',
                                   activation=None,
                                   kernel_initializer=tf.contrib.layers.xavier_initializer())
        conv_0 = tf.contrib.layers.batch_norm(conv_0, activation_fn=tf.nn.relu,is_training = is_training)      
        print conv_0
        
        
        conv_1 = tf.layers.conv2d(conv_0,
                                   filters=128/2,
                                   kernel_size = 3,
                                   strides=(1,1),
                                   padding='same',
                                   activation=None,
                                   kernel_initializer=tf.contrib.layers.xavier_initializer())
        conv_1 = tf.contrib.layers.batch_norm(conv_1, activation_fn=tf.nn.relu,is_training = is_training)
        print conv_1
        
        conv_2 = tf.layers.conv2d(conv_1,
                                   filters=256,
                                   kernel_size = 3,
                                   strides=(1,1),
                                   padding='same',
                                   activation=None,
                                   kernel_initializer=tf.contrib.layers.xavier_initializer())
        print conv_2
        
    if i==0:
        concat = tf.layers.conv2d(unactivated,
                                  filters=256,
                                  kernel_size = 1,
                                  strides=(1,1),
                                  padding='same',
                                  activation=None,
                                  kernel_initializer=tf.contrib.layers.xavier_initializer())
        
    else:
        concat = unactivated
        
    output = tf.add(conv_2,concat)
    unactivated = output
    print unactivated
    activated = tf.contrib.layers.batch_norm(output, activation_fn=tf.nn.relu,is_training = is_training)
    print activated

Tensor("block2_0/BatchNorm/Relu:0", shape=(36, 48, 48, 64), dtype=float32)
Tensor("block2_0/BatchNorm_1/Relu:0", shape=(36, 48, 48, 64), dtype=float32)
Tensor("block2_0/conv2d_2/BiasAdd:0", shape=(36, 48, 48, 256), dtype=float32)
Tensor("Add:0", shape=(36, 48, 48, 256), dtype=float32)
Tensor("BatchNorm_1/Relu:0", shape=(36, 48, 48, 256), dtype=float32)
Tensor("block2_1/BatchNorm/Relu:0", shape=(36, 48, 48, 64), dtype=float32)
Tensor("block2_1/BatchNorm_1/Relu:0", shape=(36, 48, 48, 64), dtype=float32)
Tensor("block2_1/conv2d_2/BiasAdd:0", shape=(36, 48, 48, 256), dtype=float32)
Tensor("Add_1:0", shape=(36, 48, 48, 256), dtype=float32)
Tensor("BatchNorm_2/Relu:0", shape=(36, 48, 48, 256), dtype=float32)
Tensor("block2_2/BatchNorm/Relu:0", shape=(36, 48, 48, 64), dtype=float32)
Tensor("block2_2/BatchNorm_1/Relu:0", shape=(36, 48, 48, 64), dtype=float32)
Tensor("block2_2/conv2d_2/BiasAdd:0", shape=(36, 48, 48, 256), dtype=float32)
Tensor("Add_2:0", shape=(36, 48, 48, 256), dtype=float32)


In [0]:
conv3_blocks = 3
for i in range(conv3_blocks):
    with tf.variable_scope("block3_"+str(i)):
        if i == 0:
            conv_0 = tf.layers.conv2d(activated,
                                       filters=256/2,
                                       kernel_size = 1,
                                       strides=(2,2),
                                       padding='same',
                                       activation=None,
                                       kernel_initializer=tf.contrib.layers.xavier_initializer())
            conv_0 = tf.contrib.layers.batch_norm(conv_0, activation_fn=tf.nn.relu,is_training = is_training)      
            print conv_0
        else:
            conv_0 = tf.layers.conv2d(activated,
                                       filters=256/2,
                                       kernel_size = 1,
                                       strides=(1,1),
                                       padding='same',
                                       activation=None,
                                       kernel_initializer=tf.contrib.layers.xavier_initializer())
            conv_0 = tf.contrib.layers.batch_norm(conv_0, activation_fn=tf.nn.relu,is_training = is_training)      
            print conv_0
            
        
        
        conv_1 = tf.layers.conv2d(conv_0,
                                   filters=256/2,
                                   kernel_size = 3,
                                   strides=(1,1),
                                   padding='same',
                                   activation=None,
                                   kernel_initializer=tf.contrib.layers.xavier_initializer())
        conv_1 = tf.contrib.layers.batch_norm(conv_1, activation_fn=tf.nn.relu,is_training = is_training)
        print conv_1
        
        conv_2 = tf.layers.conv2d(conv_1,
                                   filters=512,
                                   kernel_size = 3,
                                   strides=(1,1),
                                   padding='same',
                                   activation=None,
                                   kernel_initializer=tf.contrib.layers.xavier_initializer())
        print conv_2
        
    if i==0:
        concat = tf.layers.conv2d(unactivated,
                                  filters=512,
                                  kernel_size = 1,
                                  strides=(2,2),
                                  padding='same',
                                  activation=None,
                                  kernel_initializer=tf.contrib.layers.xavier_initializer())
        
    else:
        concat = unactivated
        
    output = tf.add(conv_2,concat)
    unactivated = output
    print unactivated
    activated = tf.contrib.layers.batch_norm(output, activation_fn=tf.nn.relu,is_training = is_training)
    print activated

Tensor("block3_0/BatchNorm/Relu:0", shape=(36, 24, 24, 128), dtype=float32)
Tensor("block3_0/BatchNorm_1/Relu:0", shape=(36, 24, 24, 128), dtype=float32)
Tensor("block3_0/conv2d_2/BiasAdd:0", shape=(36, 24, 24, 512), dtype=float32)
Tensor("Add_3:0", shape=(36, 24, 24, 512), dtype=float32)
Tensor("BatchNorm_4/Relu:0", shape=(36, 24, 24, 512), dtype=float32)
Tensor("block3_1/BatchNorm/Relu:0", shape=(36, 24, 24, 128), dtype=float32)
Tensor("block3_1/BatchNorm_1/Relu:0", shape=(36, 24, 24, 128), dtype=float32)
Tensor("block3_1/conv2d_2/BiasAdd:0", shape=(36, 24, 24, 512), dtype=float32)
Tensor("Add_4:0", shape=(36, 24, 24, 512), dtype=float32)
Tensor("BatchNorm_5/Relu:0", shape=(36, 24, 24, 512), dtype=float32)
Tensor("block3_2/BatchNorm/Relu:0", shape=(36, 24, 24, 128), dtype=float32)
Tensor("block3_2/BatchNorm_1/Relu:0", shape=(36, 24, 24, 128), dtype=float32)
Tensor("block3_2/conv2d_2/BiasAdd:0", shape=(36, 24, 24, 512), dtype=float32)
Tensor("Add_5:0", shape=(36, 24, 24, 512), dtype=f

In [0]:
conv4_blocks = 9
for i in range(conv4_blocks):
    with tf.variable_scope("block4_"+str(i)):
        if i == 0:
            conv_0 = tf.layers.conv2d(activated,
                                       filters=512/2,
                                       kernel_size = 1,
                                       strides=(1,1),
                                       padding='same',
                                       activation=None,
                                       kernel_initializer=tf.contrib.layers.xavier_initializer())
            conv_0 = tf.contrib.layers.batch_norm(conv_0, activation_fn=tf.nn.relu,is_training = is_training)      
            print conv_0
        else:
            conv_0 = tf.layers.conv2d(activated,
                                       filters=512/2,
                                       kernel_size = 1,
                                       strides=(1,1),
                                       padding='same',
                                       activation=None,
                                       kernel_initializer=tf.contrib.layers.xavier_initializer())
            conv_0 = tf.contrib.layers.batch_norm(conv_0, activation_fn=tf.nn.relu,is_training = is_training)      
            print conv_0
            
        
        
        conv_1 = tf.layers.conv2d(conv_0,
                                   filters=512/2,
                                   kernel_size = 3,
                                   strides=(1,1),
                                   padding='same',
                                   activation=None,
                                   kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                   dilation_rate = 2)
        conv_1 = tf.contrib.layers.batch_norm(conv_1, activation_fn=tf.nn.relu,is_training = is_training)
        print conv_1
        
        conv_2 = tf.layers.conv2d(conv_1,
                                   filters=1024,
                                   kernel_size = 3,
                                   strides=(1,1),
                                   padding='same',
                                   activation=None,
                                   kernel_initializer=tf.contrib.layers.xavier_initializer())
        print conv_2
        
    if i==0:
        concat = tf.layers.conv2d(unactivated,
                                  filters=1024,
                                  kernel_size = 1,
                                  strides=(1,1),
                                  padding='same',
                                  activation=None,
                                  kernel_initializer=tf.contrib.layers.xavier_initializer())
        
    else:
        concat = unactivated
        
    output = tf.add(conv_2,concat)
    unactivated = output
    print unactivated
    activated = tf.contrib.layers.batch_norm(output, activation_fn=tf.nn.relu,is_training = is_training)
    print activated

Tensor("block4_0/BatchNorm/Relu:0", shape=(36, 24, 24, 256), dtype=float32)
Tensor("block4_0/BatchNorm_1/Relu:0", shape=(36, 24, 24, 256), dtype=float32)
Tensor("block4_0/conv2d_2/BiasAdd:0", shape=(36, 24, 24, 1024), dtype=float32)
Tensor("Add_6:0", shape=(36, 24, 24, 1024), dtype=float32)
Tensor("BatchNorm_7/Relu:0", shape=(36, 24, 24, 1024), dtype=float32)
Tensor("block4_1/BatchNorm/Relu:0", shape=(36, 24, 24, 256), dtype=float32)
Tensor("block4_1/BatchNorm_1/Relu:0", shape=(36, 24, 24, 256), dtype=float32)
Tensor("block4_1/conv2d_2/BiasAdd:0", shape=(36, 24, 24, 1024), dtype=float32)
Tensor("Add_7:0", shape=(36, 24, 24, 1024), dtype=float32)
Tensor("BatchNorm_8/Relu:0", shape=(36, 24, 24, 1024), dtype=float32)
Tensor("block4_2/BatchNorm/Relu:0", shape=(36, 24, 24, 256), dtype=float32)
Tensor("block4_2/BatchNorm_1/Relu:0", shape=(36, 24, 24, 256), dtype=float32)
Tensor("block4_2/conv2d_2/BiasAdd:0", shape=(36, 24, 24, 1024), dtype=float32)
Tensor("Add_8:0", shape=(36, 24, 24, 1024),

In [0]:
conv5_blocks = 3
for i in range(conv5_blocks):
    with tf.variable_scope("block5_"+str(i)):
        if i == 0:
            conv_0 = tf.layers.conv2d(activated,
                                       filters=1024/2,
                                       kernel_size = 1,
                                       strides=(1,1),
                                       padding='same',
                                       activation=None,
                                       kernel_initializer=tf.contrib.layers.xavier_initializer())
            conv_0 = tf.contrib.layers.batch_norm(conv_0, activation_fn=tf.nn.relu,is_training = is_training)      
            print conv_0
        else:
            conv_0 = tf.layers.conv2d(activated,
                                       filters=1024/2,
                                       kernel_size = 1,
                                       strides=(1,1),
                                       padding='same',
                                       activation=None,
                                       kernel_initializer=tf.contrib.layers.xavier_initializer())
            conv_0 = tf.contrib.layers.batch_norm(conv_0, activation_fn=tf.nn.relu,is_training = is_training)      
            print conv_0
            
        
        
        conv_1 = tf.layers.conv2d(conv_0,
                                   filters=1024/2,
                                   kernel_size = 3,
                                   strides=(1,1),
                                   padding='same',
                                   activation=None,
                                   kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                   dilation_rate = 2)
        conv_1 = tf.contrib.layers.batch_norm(conv_1, activation_fn=tf.nn.relu,is_training = is_training)
        print conv_1
        
        conv_2 = tf.layers.conv2d(conv_1,
                                   filters=2048,
                                   kernel_size = 3,
                                   strides=(1,1),
                                   padding='same',
                                   activation=None,
                                   kernel_initializer=tf.contrib.layers.xavier_initializer())
        print conv_2
        
    if i==0:
        concat = tf.layers.conv2d(unactivated,
                                  filters=2048,
                                  kernel_size = 1,
                                  strides=(1,1),
                                  padding='same',
                                  activation=None,
                                  kernel_initializer=tf.contrib.layers.xavier_initializer())
        
    else:
        concat = unactivated
        
    output = tf.add(conv_2,concat)
    unactivated = output
    print unactivated
    activated = tf.contrib.layers.batch_norm(output, activation_fn=tf.nn.relu,is_training = is_training)
    print activated

Tensor("block5_0/BatchNorm/Relu:0", shape=(36, 24, 24, 512), dtype=float32)
Tensor("block5_0/BatchNorm_1/Relu:0", shape=(36, 24, 24, 512), dtype=float32)
Tensor("block5_0/conv2d_2/BiasAdd:0", shape=(36, 24, 24, 2048), dtype=float32)
Tensor("Add_15:0", shape=(36, 24, 24, 2048), dtype=float32)
Tensor("BatchNorm_16/Relu:0", shape=(36, 24, 24, 2048), dtype=float32)
Tensor("block5_1/BatchNorm/Relu:0", shape=(36, 24, 24, 512), dtype=float32)
Tensor("block5_1/BatchNorm_1/Relu:0", shape=(36, 24, 24, 512), dtype=float32)
Tensor("block5_1/conv2d_2/BiasAdd:0", shape=(36, 24, 24, 2048), dtype=float32)
Tensor("Add_16:0", shape=(36, 24, 24, 2048), dtype=float32)
Tensor("BatchNorm_17/Relu:0", shape=(36, 24, 24, 2048), dtype=float32)
Tensor("block5_2/BatchNorm/Relu:0", shape=(36, 24, 24, 512), dtype=float32)
Tensor("block5_2/BatchNorm_1/Relu:0", shape=(36, 24, 24, 512), dtype=float32)
Tensor("block5_2/conv2d_2/BiasAdd:0", shape=(36, 24, 24, 2048), dtype=float32)
Tensor("Add_17:0", shape=(36, 24, 24, 2

In [0]:
#ASPP with bilinear interpolation
with tf.variable_scope("prediction_and_interpolation"):
    aspp1 = tf.layers.conv2d(activated,
                                       filters=2,
                                       kernel_size = 3,
                                       strides=(1,1),
                                       padding='same',
                                       activation=None,
                                       kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                       dilation_rate = 8)
    aspp2 = tf.layers.conv2d(activated,
                                       filters=2,
                                       kernel_size = 3,
                                       strides=(1,1),
                                       padding='same',
                                       activation=None,
                                       kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                       dilation_rate = 12)
    aspp3 = tf.layers.conv2d(activated,
                                       filters=2,
                                       kernel_size = 3,
                                       strides=(1,1),
                                       padding='same',
                                       activation=None,
                                       kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                       dilation_rate = 16)
    aspp4 = tf.layers.conv2d(activated,
                                       filters=2,
                                       kernel_size = 3,
                                       strides=(1,1),
                                       padding='same',
                                       activation=None,
                                       kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                       dilation_rate = 24)
    
    pred_mask =  tf.add_n([aspp1,aspp2,aspp3,aspp4])
    
    pred_mask = tf.contrib.layers.batch_norm(pred_mask, activation_fn=tf.nn.relu,is_training = is_training)
    print pred_mask
    
    
    pred_mask = tf.image.resize_bilinear(pred_mask,size = [101,101],align_corners=True)
    print pred_mask

Tensor("prediction_and_interpolation/BatchNorm/Relu:0", shape=(36, 24, 24, 2), dtype=float32)
Tensor("prediction_and_interpolation/ResizeBilinear:0", shape=(36, 101, 101, 2), dtype=float32)


In [0]:
with tf.variable_scope("loss"):
    loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels = masks, logits = pred_mask)
    loss = tf.reduce_mean(loss)

In [0]:
minimize = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [0]:
sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)

def save_model(sess):
    !mkdir saved_model
    saver = tf.train.Saver()
    path = saver.save(sess,"./saved_model/model.ckpt")
    print "Model saved @ " + path
    return path
  
def restore_model(sess):
    saver = tf.train.Saver()
    saver.restore(sess, "./saved_model/model.ckpt")
    
def restore_model_from_drive(sess,file_id):
    downloaded = drive.CreateFile({'id': file_id})
    downloaded.GetContentFile("saved_model.tar.gz")
    !tar -xzvf saved_model.tar.gz
    restore_model(sess)

  
  
def upload_to_google_drive(filename,folder_id):
    uploaded = False
    file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(folder_id)}).GetList()
    try:
        for file1 in file_list:
            if (file1['title'] == filename) and (not uploaded):
                file1.Delete()
                f = drive.CreateFile({"parents": [{"kind": "drive#fileLink", "id": folder_id, }]})
                f.SetContentFile(filename)
                f.Upload()
                uploaded = True
        if not uploaded:
            f = drive.CreateFile({"parents": [{"kind": "drive#fileLink", "id": folder_id, }]})
            f.SetContentFile(filename)
            f.Upload()
            uploaded = True
    except:
        print "Failed"
  
    return f.get("id")

def get_one_hot_masks(masks):
    temp = (np.invert(masks.astype(dtype=bool)).astype(dtype = int))
    masks=np.expand_dims(masks,axis=3)
    temp =np.expand_dims(temp,axis=3)
    return np.concatenate([temp,masks],axis = 3)



In [0]:
def get_ious(test_images,true_masks, batch_size):
    no_of_batches = len(true_masks)/batch_size
    ptr = 0
    ious = []
    for i in range(no_of_batches):
        x = test_images[ptr:ptr+batch_size]
        y = get_one_hot_masks(true_masks[ptr:ptr+batch_size])
        prediction = sess.run(pred_mask,{image_batch:x,masks:y,is_training: True})
        prediction = np.argmax(prediction,axis=3)
        true_mask_bool = true_masks[ptr:ptr+batch_size].astype(dtype=bool)
        pred_mask_bool = prediction.astype(dtype=bool)

        for i in range(batch_size):
            temp = (true_mask_bool[i]*pred_mask_bool[i])
            intersection = np.sum(temp)
            temp = (true_mask_bool[i]+pred_mask_bool[i])
            union = np.sum(temp)
            
            if union:
                ious.append(float(intersection)/union)
            else:
                ious.append(1.0)
                
        ptr += batch_size
   
    return np.array(ious)

def train_model(train_images,train_masks,no_of_batches):
    total = 0
    ptr = 0
    print "learn_rate",learn_rate
    for j in range(no_of_batches):
        total+=1
        if j%10 == 0:
            print total
        x = train_images[ptr:ptr+batch_size]
        y = get_one_hot_masks(train_masks[ptr:ptr+batch_size])
        
        sess.run(minimize,{image_batch: x, masks: y,is_training: True,learning_rate:learn_rate})
        ptr+= batch_size
    return total

In [0]:
if train:
    folder_id = "1RTzLh1j5v2mHSQHDeJJZPHxWVfxW3xth"  #kaggle deeplab 3 folder in google drive
    filename = "saved_model.tar.gz"
    curr_iou =  0
    saved_iou= 0
    epoch = 200
    with open("validation.pkl","r") as f:
        validation_images, validation_masks= pickle.load(f)
        
    with open("train.pkl","r") as f:
        train_images , train_masks = pickle.load(f)
    
    for i in range(1,epoch+1):
        gdrive()
        print "epoch------------>>>  "+ str(i)
        batches = 0
        x = None
        y = None
        for k in range(1):
            no_of_batches = len(train_images)/batch_size
            batches = train_model(train_images,train_masks,no_of_batches)
            print batches
            x = train_images[:batch_size]
            y = get_one_hot_masks(train_masks[:batch_size])
            curr_iou = np.mean(get_ious(validation_images,validation_masks,batch_size))
            print "loss: " , sess.run(loss,{image_batch: x, masks: y,is_training: True})
            print "mean iou = " + str(curr_iou)
            if curr_iou > saved_iou:
                save_model(sess)
                !tar -czvf saved_model.tar.gz saved_model
                print upload_to_google_drive(filename,folder_id)
                saved_iou= curr_iou
            

            

epoch------------>>>  1
learn_rate 1e-05
1
11
21
31
41
51
61
71
81
91
100
loss:  0.07412914
mean iou = 0.792897898319838
epoch------------>>>  2
learn_rate 1e-05
1
11
21
31
41
51
61
71
81
91
100
loss:  0.07412495
mean iou = 0.7932571848658423
epoch------------>>>  3
learn_rate 1e-05
1
11
21
31
41
51
61
71
81
91
100
loss:  0.074102364
mean iou = 0.7933722146556325
epoch------------>>>  4
learn_rate 1e-05
1
11
21
31
41
51
61
71
81
91
100
loss:  0.07407546
mean iou = 0.7934390827585509
epoch------------>>>  5
learn_rate 1e-05
1
11


KeyboardInterrupt: ignored

In [0]:

def RLenc(img, order='F', format=True):
    """
    img is binary mask image, shape (r,c)
    order is down-then-right, i.e. Fortran
    format determines if the order needs to be preformatted (according to submission rules) or not

    returns run length as an array or string (if format is True)
    """
    bytes = img.reshape(img.shape[0] * img.shape[1], order=order)
    runs = []  ## list of run lengths
    r = 0  ## the current run length
    pos = 1  ## count starts from 1 per WK
    for c in bytes:
        if (c == 0):
            if r != 0:
                runs.append((pos, r))
                pos += r
                r = 0
            pos += 1
        else:
            r += 1

    # if last run is unsaved (i.e. data ends with 1)
    if r != 0:
        runs.append((pos, r))
        pos += r
        r = 0

    if format:
        z = ''

        for rr in runs:
            z += '{} {} '.format(rr[0], rr[1])
        return z[:-1]
    else:
        return runs

In [0]:
import csv
def add_to_submission(runs,image_ids):
    with open('submission.csv', 'a') as csvFile:
        writer=csv.writer(csvFile)
        for i in range(len(image_ids)):
            row = [image_ids[i],runs[i]]
            writer.writerow(row)
    csvFile.close()
    print "added"
            
if test:
    file_id ="1S2NP8laV8soVXNFHi-ff-Fcs5WXa2FH5"
    restore_model_from_drive(sess,file_id)
    with open("new.pkl","r") as f:
        new_images, image_ids = pickle.load(f)
    print image_ids[0]
    print new_images.shape
    predicted_masks = []
    
#     normalize new images. 3 channels seperately
    img_a = new_images[:, :, :, 0]
    img_b = new_images[:, :, :, 1]
    img_c = new_images[:, :, :, 2]

    # normalizing per channel data:
    img_a = (img_a - np.min(img_a)).astype(dtype=float) / (np.max(img_a) - np.min(img_a))
    img_b = (img_b - np.min(img_b)).astype(dtype=float) / (np.max(img_b) - np.min(img_b))
    img_c = (img_c - np.min(img_c)).astype(dtype=float) / (np.max(img_c) - np.min(img_c))

    # putting the 3 channels back together:
    new_images = np.empty(new_images.shape, dtype=np.float32)
    new_images[:, :, :, 0] = img_a
    new_images[:, :, :, 1] = img_b
    new_images[:, :, :, 2] = img_c

    del img_a,img_b,img_c

    

    header = ["id","rle_mask"]
    
    with open('submission.csv', 'w') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerow(header)

    csvFile.close()
    
    #get masks for the new images
    ptr = 0 
    no_of_batches = new_images.shape[0]/batch_size
    
    
    for i in range(no_of_batches):
        x = new_images[ptr:ptr+batch_size]        
        predicted_masks = np.argmax(sess.run(pred_mask,{image_batch:x,is_training: True}),axis=3).astype(dtype=)
        runs = []
        for i in predicted_masks:
            runs.append(RLenc(i))
        add_to_submission(runs,image_ids[ptr:ptr+batch_size])
        ptr+=batch_size

        
        
    #upload to google drive
    folder_id = "1s0ALTUOhZXdE14DRGPBQ4N7QBOu6673S" 
    upload_to_google_drive("submission.csv",folder_id)